In [1]:
import pandas as pd
import os
import random

In [2]:
idle_directory = "../data/combined data files/idle/benchmark data runs/"
attack_directory = "../data/combined data files/spectre/benchmark data runs/"

idle = "1. idle/"
cpu = "2. cpu/"
memory = "3. memory"
cpu_memory = "4. cpu + memory/"

idle_mode_idle_files = [file for file in os.listdir(idle_directory+idle)]
idle_mode_attack_files = [file for file in os.listdir(attack_directory+idle)]

cpu_mode_idle_files = [file for file in os.listdir(idle_directory+cpu)]
cpu_mode_attack_files = [file for file in os.listdir(attack_directory+cpu)]

memory_mode_idle_files = [file for file in os.listdir(idle_directory+memory)]
memory_mode_attack_files = [file for file in os.listdir(attack_directory+memory)]

cpu_memory_mode_idle_files = [file for file in os.listdir(idle_directory+cpu_memory)]
cpu_memory_mode_attack_files = [file for file in os.listdir(attack_directory+cpu_memory)]

In [3]:
executed_branch_mispredictions = "Executed Branch Mispredictions"
executed_branch_instructions = "Executed Branch Instructions"
executed_branch_misprediction_rate = "Executed Branch Misprediction Rate"

retired_branch_mispredictions = "Retired Branch Mispredictions"
retired_branch_instructions = "Retired Branch Instructions"
retired_branch_misprediction_rate = "Retired Branch Misprediction Rate"

executed_conditional_branch_mispredictions = "Executed Conditional Branch Mispredictions"
executed_conditional_branches = "Executed Conditional Branches"
executed_conditional_misprediction_rate = "Executed Conditional Misprediction Rate"

retired_near_taken_mispredictions = "Retired Near-Taken Branch Mispredictions"
retired_conditional_branches = "Retired Conditional Branches"
retired_near_taken_misprediction_rate = "Retired Near-Taken to Conditional misprediction Rate"

# Variables provided by you for L1, L2, and L3 cache metrics
L1_data_load_misses = "L1 Data Cache Load Misses"
L1_data_load_all = "L1 Data Cache Loads"
L1_data_load_miss_rate = "L1 Data Cache Load Miss Rate"

L2_demand_data_read_hits = "L2 Demand Data Read Hits"
L2_demand_data_read_all = "L2 All Demand Data Reads"
L2_demand_data_read_hit_rate = "L2 Demand Data Read Hit Ratio"

L3_demand_data_read_misses = "Offcore Demand Data Reads LLC Miss to DRAM"
L3_demand_data_read_hits = "Offcore Demand Data Reads LLC Hit Any Response"
L3_demand_data_read_miss_rate = "Offcore Demand Data Reads LLC Miss Rate"

L3_all_data_read_all = "Offcore All Data Reads LLC Any Response"
L3_all_data_read_misses = "Offcore All Data Reads LLC Miss to DRAM"
L3_all_data_read_miss_rate = "Offcore All Data Reads LLC miss rate"


branch_misses = "Branch Misses"
branch_instructions = "Branch Instructions"
branch_miss_rate = "Branch Miss Rate"

cache_misses = "Cache Misses"
cache_references = "Cache References"
cache_miss_rate = "Cache Miss Rate"

# additional rates
retired_to_executed_branch_rate = "Retired To Executed Branch Rate"
retired_to_executed_branch_misprediction_rate = "Retired to Executed Branch Misprediction Rate"

rate_pairs = [
    (L1_data_load_misses, L1_data_load_all, L1_data_load_miss_rate),
    (L2_demand_data_read_hits, L2_demand_data_read_all, L2_demand_data_read_hit_rate),
    (L3_all_data_read_misses, L3_all_data_read_all, L3_all_data_read_miss_rate),
    (executed_branch_mispredictions, executed_branch_instructions, executed_branch_misprediction_rate),
    (retired_branch_mispredictions, retired_branch_instructions, retired_branch_misprediction_rate),
    (executed_conditional_branch_mispredictions, executed_conditional_branches, executed_conditional_misprediction_rate),
    (retired_near_taken_mispredictions, retired_conditional_branches, retired_near_taken_misprediction_rate),
    (retired_branch_instructions, executed_branch_instructions, retired_to_executed_branch_rate),
    (retired_branch_mispredictions,executed_branch_mispredictions,retired_to_executed_branch_misprediction_rate),
    (branch_misses, branch_instructions, branch_miss_rate),
    (cache_misses, cache_references, cache_miss_rate),
]

def get_rate(df):
    """
    Compute logical rates from the given DataFrame based on predefined rate pairs.
    Returns a DataFrame with computed rates capped at 1 and infinities replaced with NaN.
    """
    rates_data = {}
    for numerator, denominator, rate_name in rate_pairs:
        rate = df[numerator] / df[denominator]
        rate.replace([float('inf'), -float('inf')], 0, inplace=True)  # Handle infinities
        rate = rate.clip(upper=1)  # Cap the rate at 1
        rate.fillna(0, inplace=True)
        rates_data[rate_name] = rate
    
    rate = df[L3_demand_data_read_misses] / df[L3_demand_data_read_misses]+df[L3_demand_data_read_hits]
    rate = df[numerator] / df[denominator]
    rate.replace([float('inf'), -float('inf')], 0, inplace=True)  # Handle infinities
    rate.fillna(0, inplace=True)
    rate = rate.clip(upper=1)  # Cap the rate at 1
    rates_data[L3_demand_data_read_miss_rate] = rate
    
    return pd.DataFrame(rates_data)


In [4]:
# Function to load all CSV files except the first one
def load_csv_files(directory):
    files = [file for file in os.listdir(directory) if file.endswith('.csv')]
    return {f"df_{index}": pd.read_csv(os.path.join(directory, file)) for index, file in enumerate(files[0:], start=0)}

# Load dataframes for each mode
idle_mode_idle_dataframes = load_csv_files(idle_directory + idle)
idle_mode_attack_dataframes = load_csv_files(attack_directory + idle)

cpu_mode_idle_dataframes = load_csv_files(idle_directory + cpu)
cpu_mode_attack_dataframes = load_csv_files(attack_directory + cpu)

memory_mode_idle_dataframes = load_csv_files(idle_directory + memory)
memory_mode_attack_dataframes = load_csv_files(attack_directory + memory)

cpu_memory_mode_idle_dataframes = load_csv_files(idle_directory + cpu_memory)
cpu_memory_mode_attack_dataframes = load_csv_files(attack_directory + cpu_memory)

# Display loaded dataframes
def display_dataframes_info(mode_name, dataframes):
    print(f"\n{mode_name} DataFrames:")
    for name, df in dataframes.items():
        print(f"{name}: {df.shape}")

# Display DataFrame information for all modes
display_dataframes_info("Idle Mode Idle", idle_mode_idle_dataframes)
display_dataframes_info("Idle Mode Attack", idle_mode_attack_dataframes)
display_dataframes_info("CPU Mode Idle", cpu_mode_idle_dataframes)
display_dataframes_info("CPU Mode Attack", cpu_mode_attack_dataframes)
display_dataframes_info("Memory Mode Idle", memory_mode_idle_dataframes)
display_dataframes_info("Memory Mode Attack", memory_mode_attack_dataframes)
display_dataframes_info("CPU + Memory Mode Idle", cpu_memory_mode_idle_dataframes)
display_dataframes_info("CPU + Memory Mode Attack", cpu_memory_mode_attack_dataframes)


Idle Mode Idle DataFrames:
df_0: (3003, 21)
df_1: (3002, 21)
df_2: (3003, 21)
df_3: (3003, 21)
df_4: (3003, 21)

Idle Mode Attack DataFrames:
df_0: (3349, 21)
df_1: (2383, 21)
df_2: (2767, 21)
df_3: (2736, 21)
df_4: (2985, 21)

CPU Mode Idle DataFrames:
df_0: (3005, 21)
df_1: (3006, 21)
df_2: (3005, 21)
df_3: (3007, 21)
df_4: (3005, 21)

CPU Mode Attack DataFrames:
df_0: (3112, 21)
df_1: (3515, 21)
df_2: (2968, 21)
df_3: (2938, 21)
df_4: (3502, 21)

Memory Mode Idle DataFrames:
df_0: (3015, 21)
df_1: (2412, 21)
df_2: (2407, 21)
df_3: (2411, 21)
df_4: (2414, 21)

Memory Mode Attack DataFrames:
df_0: (2060, 21)
df_1: (2628, 21)
df_2: (2190, 21)
df_3: (2325, 21)
df_4: (2347, 21)

CPU + Memory Mode Idle DataFrames:
df_0: (2410, 21)
df_1: (2405, 21)
df_2: (2410, 21)
df_3: (2405, 21)
df_4: (2406, 21)

CPU + Memory Mode Attack DataFrames:
df_0: (3291, 21)
df_1: (2787, 21)
df_2: (4972, 21)
df_3: (4014, 21)
df_4: (2981, 21)


In [5]:
# Function to trim DataFrames to the first 750 rows
def trim_dataframes(dataframes, num_rows=750):
    return {name: df.iloc[:num_rows] for name, df in dataframes.items()}

# Trim DataFrames for all modes
idle_mode_idle_dataframes = trim_dataframes(idle_mode_idle_dataframes)
idle_mode_attack_dataframes = trim_dataframes(idle_mode_attack_dataframes)

cpu_mode_idle_dataframes = trim_dataframes(cpu_mode_idle_dataframes)
cpu_mode_attack_dataframes = trim_dataframes(cpu_mode_attack_dataframes)

memory_mode_idle_dataframes = trim_dataframes(memory_mode_idle_dataframes)
memory_mode_attack_dataframes = trim_dataframes(memory_mode_attack_dataframes)

cpu_memory_mode_idle_dataframes = trim_dataframes(cpu_memory_mode_idle_dataframes)
cpu_memory_mode_attack_dataframes = trim_dataframes(cpu_memory_mode_attack_dataframes)

# Display trimmed DataFrame information
display_dataframes_info("Idle Mode Idle (Trimmed)", idle_mode_idle_dataframes)
display_dataframes_info("Idle Mode Attack (Trimmed)", idle_mode_attack_dataframes)
display_dataframes_info("CPU Mode Idle (Trimmed)", cpu_mode_idle_dataframes)
display_dataframes_info("CPU Mode Attack (Trimmed)", cpu_mode_attack_dataframes)
display_dataframes_info("Memory Mode Idle (Trimmed)", memory_mode_idle_dataframes)
display_dataframes_info("Memory Mode Attack (Trimmed)", memory_mode_attack_dataframes)
display_dataframes_info("CPU + Memory Mode Idle (Trimmed)", cpu_memory_mode_idle_dataframes)
display_dataframes_info("CPU + Memory Mode Attack (Trimmed)", cpu_memory_mode_attack_dataframes)



Idle Mode Idle (Trimmed) DataFrames:
df_0: (750, 21)
df_1: (750, 21)
df_2: (750, 21)
df_3: (750, 21)
df_4: (750, 21)

Idle Mode Attack (Trimmed) DataFrames:
df_0: (750, 21)
df_1: (750, 21)
df_2: (750, 21)
df_3: (750, 21)
df_4: (750, 21)

CPU Mode Idle (Trimmed) DataFrames:
df_0: (750, 21)
df_1: (750, 21)
df_2: (750, 21)
df_3: (750, 21)
df_4: (750, 21)

CPU Mode Attack (Trimmed) DataFrames:
df_0: (750, 21)
df_1: (750, 21)
df_2: (750, 21)
df_3: (750, 21)
df_4: (750, 21)

Memory Mode Idle (Trimmed) DataFrames:
df_0: (750, 21)
df_1: (750, 21)
df_2: (750, 21)
df_3: (750, 21)
df_4: (750, 21)

Memory Mode Attack (Trimmed) DataFrames:
df_0: (750, 21)
df_1: (750, 21)
df_2: (750, 21)
df_3: (750, 21)
df_4: (750, 21)

CPU + Memory Mode Idle (Trimmed) DataFrames:
df_0: (750, 21)
df_1: (750, 21)
df_2: (750, 21)
df_3: (750, 21)
df_4: (750, 21)

CPU + Memory Mode Attack (Trimmed) DataFrames:
df_0: (750, 21)
df_1: (750, 21)
df_2: (750, 21)
df_3: (750, 21)
df_4: (750, 21)


In [6]:
# Function to concatenate DataFrames in a dictionary into a single DataFrame
def concatenate_dataframes(dataframes):
    return pd.concat(dataframes.values(), ignore_index=True)

# Create concatenated DataFrames for each mode
idle_mode_idle_concat = concatenate_dataframes(idle_mode_idle_dataframes)
idle_mode_attack_concat = concatenate_dataframes(idle_mode_attack_dataframes)

cpu_mode_idle_concat = concatenate_dataframes(cpu_mode_idle_dataframes)
cpu_mode_attack_concat = concatenate_dataframes(cpu_mode_attack_dataframes)

memory_mode_idle_concat = concatenate_dataframes(memory_mode_idle_dataframes)
memory_mode_attack_concat = concatenate_dataframes(memory_mode_attack_dataframes)

cpu_memory_mode_idle_concat = concatenate_dataframes(cpu_memory_mode_idle_dataframes)
cpu_memory_mode_attack_concat = concatenate_dataframes(cpu_memory_mode_attack_dataframes)

In [7]:
# Function to calculate average values for a DataFrame
def calculate_average_row(df):
    return df.mean(axis=0)[1:21]

# Create a dictionary to store the averages for each concatenated DataFrame
average_rows = {
    "Idle Mode Idle": calculate_average_row(idle_mode_idle_concat),
    "Idle Mode Attack": calculate_average_row(idle_mode_attack_concat),
    "CPU Mode Idle": calculate_average_row(cpu_mode_idle_concat),
    "CPU Mode Attack": calculate_average_row(cpu_mode_attack_concat),
    "Memory Mode Idle": calculate_average_row(memory_mode_idle_concat),
    "Memory Mode Attack": calculate_average_row(memory_mode_attack_concat),
    "CPU + Memory Mode Idle": calculate_average_row(cpu_memory_mode_idle_concat),
    "CPU + Memory Mode Attack": calculate_average_row(cpu_memory_mode_attack_concat),
}

# Convert the dictionary of averages into a single DataFrame
average_summary_df = pd.DataFrame(average_rows).T  # Transpose to make rows correspond to modes
# average_summary_df.columns = [f"Average_{col}" for col in idle_mode_idle_concat.columns]  # Rename columns

# Display the resulting DataFrame
print("\nSummary DataFrame with Averages:")
print(average_summary_df)



Summary DataFrame with Averages:
                          Branch Instructions  Branch Misses   Cache Misses  \
Idle Mode Idle                   7.765279e+04    2675.304800    2738.784533   
Idle Mode Attack                 8.564266e+05    9504.496533   38063.767733   
CPU Mode Idle                    3.911820e+06   10567.501067    1778.806933   
CPU Mode Attack                  3.918355e+06   12767.041333   22537.664000   
Memory Mode Idle                 7.653283e+05    4415.405867  230219.889333   
Memory Mode Attack               8.562785e+05    5757.346933  216797.134400   
CPU + Memory Mode Idle           2.901047e+06  103117.061867   12286.209067   
CPU + Memory Mode Attack         3.382581e+06  120035.700533   38637.532000   

                          Cache References  Executed Branch Instructions  \
Idle Mode Idle                 8488.557333                  21689.786667   
Idle Mode Attack              48074.585333                  25288.480000   
CPU Mode Idle             

In [8]:
# Transpose the DataFrame
transposed_df = average_summary_df.T

# Convert the transposed DataFrame to LaTeX
latex_table_transposed = transposed_df.to_latex(index=True, float_format="%.2f", 
                                                caption="Transposed Average Values for Each Mode", 
                                                label="tab:transposed_average_summary")

# Save to a file (optional)
with open("transposed_average_summary_table.tex", "w") as f:
    f.write(latex_table_transposed)

# Print the transposed LaTeX table
print(latex_table_transposed)


\begin{table}
\caption{Transposed Average Values for Each Mode}
\label{tab:transposed_average_summary}
\begin{tabular}{lrrrrrrrr}
\toprule
 & Idle Mode Idle & Idle Mode Attack & CPU Mode Idle & CPU Mode Attack & Memory Mode Idle & Memory Mode Attack & CPU + Memory Mode Idle & CPU + Memory Mode Attack \\
\midrule
Branch Instructions & 77652.79 & 856426.64 & 3911820.28 & 3918355.43 & 765328.34 & 856278.53 & 2901046.85 & 3382581.35 \\
Branch Misses & 2675.30 & 9504.50 & 10567.50 & 12767.04 & 4415.41 & 5757.35 & 103117.06 & 120035.70 \\
Cache Misses & 2738.78 & 38063.77 & 1778.81 & 22537.66 & 230219.89 & 216797.13 & 12286.21 & 38637.53 \\
Cache References & 8488.56 & 48074.59 & 59267.11 & 69105.41 & 397942.51 & 389978.75 & 238056.42 & 254241.02 \\
Executed Branch Instructions & 21689.79 & 25288.48 & 25665.07 & 25771.15 & 24801.95 & 24500.35 & 24788.53 & 25209.60 \\
Executed Branch Mispredictions & 2046.51 & 8091.01 & 12737.49 & 14553.09 & 4333.47 & 6682.05 & 40574.24 & 36055.39 \\
Executed